# Import

In [2]:
import os
import shutil
import random
import pandas as pd
import json
import cv2
import zipfile
import Augmentor
import albumentations as A
import torch

from ultralytics import YOLO
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score
from IPython.display import Image, display
from keras.preprocessing.image import ImageDataGenerator
from fastai.vision.all import *




2023-11-20 14:59:54.975732: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 14:59:54.975767: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 14:59:54.975784: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Download

In [ ]:
!mkdir -p ~/.kaggle
!echo '{"username":"nazurawirayudatama","key":"bfbccdbc0ce334c8309246ba9117eb26"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download stevepython/osteoporosis-knee-xray-dataset

In [ ]:
!mv ./data/normal/normal/* ./data/normal/
!mv ./data/osteoporosis/osteoporosis/* ./data/osteoporosis/
!rmdir ./data/normal/normal
!rmdir ./data/osteoporosis/osteoporosis

# Unzipping

In [9]:
zip_file_path = '/home/nazura101/Kuliah/PCD/osteoporosis-knee-xray-dataset.zip'
extract_folder = '/home/nazura101/Kuliah/PCD/data'  # Ganti dengan path folder tujuan ekstraksi

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print(f"File ZIP telah diekstrak ke folder: {extract_folder}")

File ZIP telah diekstrak ke folder: /home/nazura101/Kuliah/PCD/data


# Directory Adjustment

In [10]:
!mv ./data/normal/normal/* ./data/normal/
!mv ./data/osteoporosis/osteoporosis/* ./data/osteoporosis/
!rmdir ./data/normal/normal
!rmdir ./data/osteoporosis/osteoporosis

# Data

In [19]:
!rm -r /home/nazura101/Kuliah/PCD/runs

In [20]:
!rm -r /home/nazura101/Kuliah/PCD/splitted_data

In [25]:
def split_directory(src_dir, train_dir, val_dir, test_dir, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1, random_seed=42):
    # Set a random seed for reproducibility
    random.seed(random_seed)

    # Recursively walk through the source directory
    for root, dirs, files in os.walk(src_dir):
        for file in files:
            src_file = os.path.join(root, file)

            # Determine whether this file goes to the train, val, or test set
            rand_value = random.random()
            if rand_value < train_ratio:
                dest_file = os.path.join(train_dir, os.path.relpath(src_file, src_dir))
            elif rand_value < train_ratio + val_ratio:
                dest_file = os.path.join(val_dir, os.path.relpath(src_file, src_dir))
            else:
                dest_file = os.path.join(test_dir, os.path.relpath(src_file, src_dir))

            # Ensure the destination directory structure exists
            os.makedirs(os.path.dirname(dest_file), exist_ok=True)

            # Copy the file to its destination
            shutil.copy(src_file, dest_file)

def augment_data(source_directory, num_samples):
    # Initialize Augmentor pipeline
    pipeline = Augmentor.Pipeline(source_directory)
    pipeline.flip_left_right(probability=0.5)
    pipeline.random_contrast(probability=0.5, min_factor=0.8, max_factor=1.2)
    pipeline.random_brightness(probability=0.5, min_factor=0.8, max_factor=1.2)

    # Augment the data
    pipeline.sample(num_samples)

def create_data_loaders(data_dir, batch_size=32):
    # Define the transformation for the images (you may need to modify this based on your data)
    item_tfms = [Resize(512), ToTensor()]
    batch_tfms = [FlipItem(p=0.5)]

    # Create data loaders
    data = ImageDataLoaders.from_folder(data_dir, train='train', valid='val', item_tfms=item_tfms, batch_tfms=batch_tfms, bs=batch_size, num_workers=0)
    
    return data

if __name__ == "__main__":
    source_directory = "/home/nazura101/Kuliah/PCD/data/"
    train_ratio = 0.7
    val_ratio = 0.2
    test_ratio = 0.1
    num_samples = 3000
    random_seed = 42

    # Split the original dataset into train, val, and test
    train_directory = "splitted_data/train"
    val_directory = "splitted_data/val"
    test_directory = "splitted_data/test"
    split_directory(source_directory, train_directory, val_directory, test_directory, train_ratio, val_ratio, random_seed)
    print(f"Split dataset into train, val, and test with ratios {train_ratio}:{val_ratio}:{test_ratio}.")

    # Augment the training data
    augment_data(train_directory, num_samples)
    print(f"Augmented {num_samples} samples")

    # Create DataLoaders for fastai
    data = create_data_loaders("splitted_data", batch_size=32)

    # Now you can use data for training your model with fastai


Split dataset into train, val, and test with ratios 0.7:0.2:0.1.
Initialised with 252 image(s) found.
Output directory set to splitted_data/train/output.

Processing <PIL.Image.Image image mode=RGB size=472x1024>: 100%|██████████| 3000/3000 [02:41<00:00, 18.52 Samples/s]                   


Augmented 3000 samples


In [26]:
# Path folder asal dan tujuan
src_path_normal = "/home/nazura101/Kuliah/PCD/splitted_data/train/output/normal/*"
src_path_osteoporosis = "/home/nazura101/Kuliah/PCD/splitted_data/train/output/osteoporosis/*"
dst_path_normal = "/home/nazura101/Kuliah/PCD/splitted_data/train/normal"
dst_path_osteoporosis = "/home/nazura101/Kuliah/PCD/splitted_data/train/osteoporosis"

# Memindahkan folder "normal" dan menimpa jika sudah ada
!mv -f $src_path_normal $dst_path_normal

# Memindahkan folder "osteoporosis" dan menimpa jika sudah ada
!mv -f $src_path_osteoporosis $dst_path_osteoporosis

In [27]:
!rm -r /home/nazura101/Kuliah/PCD/splitted_data/train/output

In [28]:
model = YOLO('yolov8n-cls.pt')
result = model.train(data = "splitted_data", epochs=100, patience=20, imgsz=512, close_mosaic=0, degrees= 10, fliplr=0.3, scale=0.3, translate=0.3, plots= True)

New https://pypi.org/project/ultralytics/8.0.210 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.180 🚀 Python-3.10.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060, 12044MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=splitted_data, epochs=100, patience=20, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=F

In [29]:
model = YOLO('yolov8s-cls.pt')
result = model.train(data = "splitted_data", epochs=100, patience=20, imgsz=512, close_mosaic=0, degrees= 10, fliplr=0.3, scale=0.3, translate=0.3, plots= True)

100%|██████████| 12.2M/12.2M [00:00<00:00, 43.3MB/s]
New https://pypi.org/project/ultralytics/8.0.210 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.180 🚀 Python-3.10.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060, 12044MiB)
engine/trainer: task=classify, mode=train, model=yolov8s-cls.pt, data=splitted_data, epochs=100, patience=20, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buff

In [30]:
model = YOLO('yolov8m-cls.pt')
result = model.train(data = "splitted_data", epochs=100, patience=20, imgsz=512, close_mosaic=0, degrees= 10, fliplr=0.3, scale=0.3, translate=0.3, plots= True)

100%|██████████| 32.7M/32.7M [00:03<00:00, 10.9MB/s]
New https://pypi.org/project/ultralytics/8.0.210 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.180 🚀 Python-3.10.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060, 12044MiB)
engine/trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=splitted_data, epochs=100, patience=20, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buff

In [31]:
model = YOLO('yolov8l-cls.pt')
result = model.train(data = "splitted_data", epochs=100, patience=20, imgsz=512, close_mosaic=0, degrees= 10, fliplr=0.3, scale=0.3, translate=0.3, plots= True)

100%|██████████| 71.7M/71.7M [00:01<00:00, 69.1MB/s]
New https://pypi.org/project/ultralytics/8.0.210 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.180 🚀 Python-3.10.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060, 12044MiB)
engine/trainer: task=classify, mode=train, model=yolov8l-cls.pt, data=splitted_data, epochs=100, patience=20, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buff

In [32]:
model = YOLO('yolov8x-cls.pt')
result = model.train(data = "splitted_data", epochs=100, patience=20, imgsz=512, close_mosaic=0, degrees= 10, fliplr=0.3, scale=0.3, translate=0.3, plots= True)

100%|██████████| 110M/110M [00:01<00:00, 78.2MB/s] 
New https://pypi.org/project/ultralytics/8.0.210 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.180 🚀 Python-3.10.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060, 12044MiB)
engine/trainer: task=classify, mode=train, model=yolov8x-cls.pt, data=splitted_data, epochs=100, patience=20, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffe

# Get Best Model

In [8]:
bestModel1 = YOLO('/home/nazura101/Kuliah/PCD/runs/classify/train/weights/best.pt')  # Gantilah dengan nama file yang benar

In [9]:
bestModel2 = YOLO('/home/nazura101/Kuliah/PCD/runs/classify/train2/weights/best.pt')  # Gantilah dengan nama file yang benar

In [10]:
bestModel3 = YOLO('/home/nazura101/Kuliah/PCD/runs/classify/train/weights/best.pt')  # Gantilah dengan nama file yang benar

In [11]:
bestModel4 = YOLO('/home/nazura101/Kuliah/PCD/runs/classify/train/weights/best.pt')  # Gantilah dengan nama file yang benar

In [12]:
bestModel5 = YOLO('/home/nazura101/Kuliah/PCD/runs/classify/train/weights/best.pt')  # Gantilah dengan nama file yang benar

In [13]:
#getting metrics (on val and test) on best model

metrics_test1 = bestModel1.val(data="splitted_data", split="test")

Ultralytics YOLOv8.0.180 🚀 Python-3.10.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060, 12044MiB)
YOLOv8n-cls summary (fused): 73 layers, 1437442 parameters, 0 gradients, 3.3 GFLOPs
train: /home/nazura101/Kuliah/PCD/splitted_data/train... found 3252 images in 2 classes ✅ 
val: /home/nazura101/Kuliah/PCD/splitted_data/val... found 77 images in 2 classes ✅ 
test: /home/nazura101/Kuliah/PCD/splitted_data/test... found 43 images in 2 classes ✅ 
test: Scanning /home/nazura101/Kuliah/PCD/splitted_data/test... 43 images, 0 corrupt: 100%|██████████| 43/43 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 3/3 [00:01<00:00,  2.39it/s]
                   all      0.744          1
Speed: 1.5ms preprocess, 12.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/val


In [14]:
metrics_test2 = bestModel2.val(data="splitted_data", split="test")

Ultralytics YOLOv8.0.180 🚀 Python-3.10.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060, 12044MiB)
YOLOv8s-cls summary (fused): 73 layers, 5077762 parameters, 0 gradients, 12.4 GFLOPs
train: /home/nazura101/Kuliah/PCD/splitted_data/train... found 3252 images in 2 classes ✅ 
val: /home/nazura101/Kuliah/PCD/splitted_data/val... found 77 images in 2 classes ✅ 
test: /home/nazura101/Kuliah/PCD/splitted_data/test... found 43 images in 2 classes ✅ 
test: Scanning /home/nazura101/Kuliah/PCD/splitted_data/test... 43 images, 0 corrupt: 100%|██████████| 43/43 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 3/3 [00:01<00:00,  2.87it/s]
                   all      0.767          1
Speed: 3.0ms preprocess, 9.9ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/val2


In [15]:
metrics_test3 = bestModel3.val(data="splitted_data", split="test")

Ultralytics YOLOv8.0.180 🚀 Python-3.10.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060, 12044MiB)
YOLOv8n-cls summary (fused): 73 layers, 1437442 parameters, 0 gradients, 3.3 GFLOPs
train: /home/nazura101/Kuliah/PCD/splitted_data/train... found 3252 images in 2 classes ✅ 
val: /home/nazura101/Kuliah/PCD/splitted_data/val... found 77 images in 2 classes ✅ 
test: /home/nazura101/Kuliah/PCD/splitted_data/test... found 43 images in 2 classes ✅ 
test: Scanning /home/nazura101/Kuliah/PCD/splitted_data/test... 43 images, 0 corrupt: 100%|██████████| 43/43 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 3/3 [00:00<00:00,  3.84it/s]
                   all      0.744          1
Speed: 3.4ms preprocess, 2.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/val3


In [16]:
metrics_test4 = bestModel4.val(data="splitted_data", split="test")

Ultralytics YOLOv8.0.180 🚀 Python-3.10.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060, 12044MiB)
YOLOv8n-cls summary (fused): 73 layers, 1437442 parameters, 0 gradients, 3.3 GFLOPs
train: /home/nazura101/Kuliah/PCD/splitted_data/train... found 3252 images in 2 classes ✅ 
val: /home/nazura101/Kuliah/PCD/splitted_data/val... found 77 images in 2 classes ✅ 
test: /home/nazura101/Kuliah/PCD/splitted_data/test... found 43 images in 2 classes ✅ 
test: Scanning /home/nazura101/Kuliah/PCD/splitted_data/test... 43 images, 0 corrupt: 100%|██████████| 43/43 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 3/3 [00:00<00:00,  3.78it/s]
                   all      0.744          1
Speed: 3.4ms preprocess, 0.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/val4


In [17]:
metrics_test5 = bestModel5.val(data="splitted_data", split="test")

Ultralytics YOLOv8.0.180 🚀 Python-3.10.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060, 12044MiB)
YOLOv8n-cls summary (fused): 73 layers, 1437442 parameters, 0 gradients, 3.3 GFLOPs
train: /home/nazura101/Kuliah/PCD/splitted_data/train... found 3252 images in 2 classes ✅ 
val: /home/nazura101/Kuliah/PCD/splitted_data/val... found 77 images in 2 classes ✅ 
test: /home/nazura101/Kuliah/PCD/splitted_data/test... found 43 images in 2 classes ✅ 
test: Scanning /home/nazura101/Kuliah/PCD/splitted_data/test... 43 images, 0 corrupt: 100%|██████████| 43/43 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 3/3 [00:00<00:00,  3.34it/s]
                   all      0.744          1
Speed: 3.9ms preprocess, 3.8ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/val5


# Evaluasi

In [24]:
cf_matrix1 = metrics_test1.confusion_matrix.matrix

In [25]:
cf_matrix2 = metrics_test2.confusion_matrix.matrix

In [26]:
cf_matrix3 = metrics_test3.confusion_matrix.matrix

In [27]:
cf_matrix4 = metrics_test4.confusion_matrix.matrix

In [28]:
cf_matrix5 = metrics_test5.confusion_matrix.matrix

In [29]:
classes = ['Normal', 'Osteoporosis']

TP_arr, FP_arr, FN_arr = [], [], []

rows, cols = cf_matrix1.shape

for i in range(rows):
    tp = cf_matrix1[i, i]
    fp = sum(cf_matrix1[i, :]) - tp
    fn = sum(cf_matrix1[:, i]) - tp
    TP_arr.append(tp)
    FP_arr.append(fp)
    FN_arr.append(fn)

precision_arr = [tp / (tp + fp) for tp, fp in zip(TP_arr, FP_arr)]
recall_arr = [tp / (tp + fn) for tp, fn in zip(TP_arr, FN_arr)]
f1_arr = [2 * (p * r) / (p + r) for p, r in zip(precision_arr, recall_arr)]

for cl, p, r, f1 in zip(classes, precision_arr, recall_arr, f1_arr):
    print(f'{cl}- ')
    print(f'Precision : {p:.2f}')
    print(f'Recall : {r:.2f}')
    print(f'F1-Score : {f1:.2f}\n')

Normal- 
Precision : 1.00
Recall : 0.58
F1-Score : 0.73

Osteoporosis- 
Precision : 0.61
Recall : 1.00
F1-Score : 0.76



In [30]:
classes = ['Normal', 'Osteoporosis']

TP_arr, FP_arr, FN_arr = [], [], []

rows, cols = cf_matrix2.shape

for i in range(rows):
    tp = cf_matrix2[i, i]
    fp = sum(cf_matrix2[i, :]) - tp
    fn = sum(cf_matrix2[:, i]) - tp
    TP_arr.append(tp)
    FP_arr.append(fp)
    FN_arr.append(fn)

precision_arr = [tp / (tp + fp) for tp, fp in zip(TP_arr, FP_arr)]
recall_arr = [tp / (tp + fn) for tp, fn in zip(TP_arr, FN_arr)]
f1_arr = [2 * (p * r) / (p + r) for p, r in zip(precision_arr, recall_arr)]

for cl, p, r, f1 in zip(classes, precision_arr, recall_arr, f1_arr):
    print(f'{cl}- ')
    print(f'Precision : {p:.2f}')
    print(f'Recall : {r:.2f}')
    print(f'F1-Score : {f1:.2f}\n')

Normal- 
Precision : 1.00
Recall : 0.62
F1-Score : 0.76

Osteoporosis- 
Precision : 0.63
Recall : 1.00
F1-Score : 0.77



In [31]:
classes = ['Normal', 'Osteoporosis']

TP_arr, FP_arr, FN_arr = [], [], []

rows, cols = cf_matrix3.shape

for i in range(rows):
    tp = cf_matrix3[i, i]
    fp = sum(cf_matrix3[i, :]) - tp
    fn = sum(cf_matrix3[:, i]) - tp
    TP_arr.append(tp)
    FP_arr.append(fp)
    FN_arr.append(fn)

precision_arr = [tp / (tp + fp) for tp, fp in zip(TP_arr, FP_arr)]
recall_arr = [tp / (tp + fn) for tp, fn in zip(TP_arr, FN_arr)]
f1_arr = [2 * (p * r) / (p + r) for p, r in zip(precision_arr, recall_arr)]

for cl, p, r, f1 in zip(classes, precision_arr, recall_arr, f1_arr):
    print(f'{cl}- ')
    print(f'Precision : {p:.2f}')
    print(f'Recall : {r:.2f}')
    print(f'F1-Score : {f1:.2f}\n')

Normal- 
Precision : 1.00
Recall : 0.58
F1-Score : 0.73

Osteoporosis- 
Precision : 0.61
Recall : 1.00
F1-Score : 0.76



In [32]:
classes = ['Normal', 'Osteoporosis']

TP_arr, FP_arr, FN_arr = [], [], []

rows, cols = cf_matrix4.shape

for i in range(rows):
    tp = cf_matrix4[i, i]
    fp = sum(cf_matrix4[i, :]) - tp
    fn = sum(cf_matrix4[:, i]) - tp
    TP_arr.append(tp)
    FP_arr.append(fp)
    FN_arr.append(fn)

precision_arr = [tp / (tp + fp) for tp, fp in zip(TP_arr, FP_arr)]
recall_arr = [tp / (tp + fn) for tp, fn in zip(TP_arr, FN_arr)]
f1_arr = [2 * (p * r) / (p + r) for p, r in zip(precision_arr, recall_arr)]

for cl, p, r, f1 in zip(classes, precision_arr, recall_arr, f1_arr):
    print(f'{cl}- ')
    print(f'Precision : {p:.2f}')
    print(f'Recall : {r:.2f}')
    print(f'F1-Score : {f1:.2f}\n')

Normal- 
Precision : 1.00
Recall : 0.58
F1-Score : 0.73

Osteoporosis- 
Precision : 0.61
Recall : 1.00
F1-Score : 0.76



In [33]:
classes = ['Normal', 'Osteoporosis']

TP_arr, FP_arr, FN_arr = [], [], []

rows, cols = cf_matrix5.shape

for i in range(rows):
    tp = cf_matrix5[i, i]
    fp = sum(cf_matrix5[i, :]) - tp
    fn = sum(cf_matrix5[:, i]) - tp
    TP_arr.append(tp)
    FP_arr.append(fp)
    FN_arr.append(fn)

precision_arr = [tp / (tp + fp) for tp, fp in zip(TP_arr, FP_arr)]
recall_arr = [tp / (tp + fn) for tp, fn in zip(TP_arr, FN_arr)]
f1_arr = [2 * (p * r) / (p + r) for p, r in zip(precision_arr, recall_arr)]

for cl, p, r, f1 in zip(classes, precision_arr, recall_arr, f1_arr):
    print(f'{cl}- ')
    print(f'Precision : {p:.2f}')
    print(f'Recall : {r:.2f}')
    print(f'F1-Score : {f1:.2f}\n')

Normal- 
Precision : 1.00
Recall : 0.58
F1-Score : 0.73

Osteoporosis- 
Precision : 0.61
Recall : 1.00
F1-Score : 0.76



# Model YOLOv8

In [3]:
model1 = YOLO('yolov8n-cls.pt')
model2 = YOLO('yolov8s-cls.pt')
model3 = YOLO('yolov8m-cls.pt')
model4 = YOLO('yolov8l-cls.pt')
model5 = YOLO('yolov8x-cls.pt')

In [8]:
model1

YOLO(
  (model): ClassificationModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_

In [7]:
model2

YOLO(
  (model): ClassificationModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_

In [6]:
model3

YOLO(
  (model): ClassificationModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running

In [5]:
model4

YOLO(
  (model): ClassificationModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(320, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_

In [4]:
model5

YOLO(
  (model): ClassificationModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 80, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(160, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(400, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_